### set up

In [1]:
from src import *
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [2]:
drive_path = r'/Users/maoqingwang/Library/CloudStorage/GoogleDrive-mw3827@columbia.edu/Shared drives/' 
base_path = 'ColumbiaProject2025/RunsWithTimeStamps'
run_dir = os.path.join(drive_path, base_path)
directory = '20250827_1128_Nasdaq_1530'

### data

In [3]:
df = read_parquet_dir(os.path.join(run_dir, directory), daily = True, price = True, begin = pd.Timestamp('15:30'), end = pd.Timestamp('16:00'))

In [4]:
print("Columns in df:", df.columns.tolist()[:20])  # First 20 columns
print("Has price column:", 'price' in df.columns)
print("DataFrame shape:", df.shape)
df.head()

Columns in df: ['0_long', '1_long', '2_short', '3_short', '4_long', '5_long', '6_long', '7_long', '8_long', '9_short', '10_short', '11_short', '12_long', '13_long', '14_long', '15_short', '16_long', '17_long', '18_long', '19_short']
Has price column: True
DataFrame shape: (10190, 268)


,0_long,1_long,2_short,3_short,4_long,5_long,6_long,7_long,8_long,9_short,...,258_long,259_long,260_long,261_long,262_short,263_short,264_short,265_short,naive_ave_pnl,price
timestamp,,,,,,,,,,,,,,,,,,,,,
2006-01-02 15:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1654.5
2006-01-02 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1654.5
2006-01-03 15:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1697.0
2006-01-03 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1693.0
2006-01-04 15:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1707.0


In [5]:
def excess_return(df: pd.DataFrame):
    import numpy as np
    cols = [c for c in df.columns if c!='price']

    position = np.array([-1 if 'short' in c else 1 for c in cols])

    L = min(len(df.iloc[1::2]), len(df.iloc[::2]))

    if L == 0:
        # fallback – keep shape consistent
        return pd.DataFrame(index=df.index, columns=cols)

    # trade-level pnl = every two rows diff
    df1 = (
        df.iloc[1:1+2*L:2].reset_index(drop=True) 
        - df.iloc[:2*L:2].reset_index(drop=True)
    )

    # align timestamp to the exit rows
    df1.index = df.index[1:1+2*L:2]
   

    market = df1['price'].cumsum()
    pnl = df1[cols].cumsum()

    traded_days = (df1[cols]!=0).astype(int).cumsum().replace(0,np.nan)
    market_days = ((market!=0).astype(int)).cumsum().replace(0,np.nan)

    ret = pnl/traded_days - (market.values.reshape(-1,1)/market_days.values.reshape(-1,1))*position
    return ret


In [6]:
ret = excess_return(df)
ret.fillna(0, inplace=True)
ret

,0_long,1_long,2_short,3_short,4_long,5_long,6_long,7_long,8_long,9_short,...,257_long,258_long,259_long,260_long,261_long,262_short,263_short,264_short,265_short,naive_ave_pnl
timestamp,,,,,,,,,,,,,,,,,,,,,
2006-01-02 16:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2006-01-03 16:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2006-01-04 16:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2006-01-05 16:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2006-01-06 16:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-07 16:00:00,19.184159,9.677367,23.899386,37.255574,29.472367,15.536829,13.740224,46.520444,8.128617,43.948318,...,12.886187,10.668795,44.870094,30.003027,3.915549,44.301235,29.330219,52.674785,35.478263,2.172397
2025-07-08 16:00:00,19.183256,9.676464,23.900290,37.256478,29.471464,15.535926,13.739321,46.519541,8.172019,43.949222,...,12.885284,10.667892,44.274241,29.890856,3.914645,44.302138,29.331123,52.675688,35.479167,2.171079
2025-07-09 16:00:00,19.179800,9.673007,23.903746,37.259934,29.468007,15.532469,13.735864,47.389304,8.442565,43.952678,...,12.881827,10.664436,44.270785,30.259674,4.507422,44.305594,29.334579,52.679145,35.482623,2.168143


In [7]:
df1 = df.iloc[1::2]
df1 = df1.drop(columns=['price'])
df1.index = df.index[1::2].date

df1

,0_long,1_long,2_short,3_short,4_long,5_long,6_long,7_long,8_long,9_short,...,257_long,258_long,259_long,260_long,261_long,262_short,263_short,264_short,265_short,naive_ave_pnl
2006-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2006-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-07,5058.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1207.0,1799.0,8677.0,...,2059.0,518.0,1970.0,3170.0,420.0,6349.0,4267.0,4169.0,8467.0,3266.097744
2025-07-08,5058.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1207.0,1817.0,8677.0,...,2059.0,518.0,1988.0,3188.0,420.0,6349.0,4267.0,4169.0,8467.0,3267.518797
2025-07-09,5058.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1277.0,1887.0,8677.0,...,2059.0,518.0,1988.0,3258.0,490.0,6349.0,4267.0,4169.0,8467.0,3270.413534
2025-07-10,4993.0,3353.0,7391.0,1270.0,2350.0,3443.0,1146.0,1277.0,1822.0,8677.0,...,2059.0,518.0,1988.0,3193.0,425.0,6349.0,4267.0,4169.0,8467.0,3265.281955


## simple ranking

PCA weighting

In [11]:
# cumpnl = df[col]                 # 已经是 cumulative
# pnl_daily = df[col].diff()       # 这才是 daily pnl
# returns = pnl_daily              # returns = daily PnL（没有 price 不要 pct_change）


In [8]:
# Import the calc_metrics function from the metrics module
from src.metrics import calc_metrics




In [9]:
# Let's run the full computation with error handling
price = df['price']
files = [c for c in df.columns if c not in ['price', 'naive_ave_pnl']]


def build_metric_matrix(df: pd.DataFrame, excess_return_func):
    price = df['price']
    files = [c for c in df.columns if c not in ['price','naive_ave_pnl']]

    ex_df = excess_return_func(df)   # 你已经有这个函数

    rows=[]
    for col in files:
        cumpnl    = df[col]
        pnl_daily = df[col].diff()

        vals = calc_metrics(cumpnl, pnl_daily, price, ex_df, col)
        rows.append(vals)

    X = pd.DataFrame.from_records(rows, index=files)
    return X


In [14]:
X = build_metric_matrix(df, excess_return)
print(X.head())
print(X.std())


         mdd_percent  ldd_percent    sharpe  cdd_percent  mdd_days  ldd_days  \
0_long      0.091328     0.196675  0.531823     0.013018        85       982   
1_long      0.284223     0.388905  0.261342     0.284223         1      1304   
2_short     0.228927     0.218103  0.356983     0.000000       737      1612   
3_short     0.255906     1.225197  0.169999     0.204724       129      1556   
4_long      0.186383     0.594894  0.401678     0.000000         1      1398   

          pnl_t  days_since_first_trade  drawdown_beta  drawup_beta  \
0_long   4993.0                   10190     -55.593943  -106.039122   
1_long   3353.0                   10190     -60.185842  -116.527419   
2_short  7391.0                   10190     -19.912190     6.862248   
3_short  1270.0                   10190     -21.132478    12.244520   
4_long   2350.0                   10190      -5.931915   -33.735332   

         excess_mean_ret  excess_ret  cvar_5  success_rate  
0_long        123.489646   18.8

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

# normalize helper
def _normalize_weights(w: pd.Series):
    w = w.fillna(0.0)
    s = w.sum()
    return w if s==0 else (w/s)


def shrink_to_equal(w_data: pd.Series, lam=0.3):
    w_eq = pd.Series(1.0/len(w_data), index=w_data.index)
    w = (1-lam)*w_eq + lam*w_data
    return _normalize_weights(w)


def pca_metric_weights_relaxed(X: pd.DataFrame, ncomp=4):
    Xc = X.copy()

    # median fill instead of drop
    Xc = Xc.fillna(Xc.median(numeric_only=True))

    # drop constant-col (zero std)
    std = Xc.std()
    keep = std[std>1e-12].index
    Xc = Xc[keep]

    if Xc.shape[1]==0:
        print("WARNING: all metrics constant → fallback equal-weight")
        return _normalize_weights(pd.Series(1.0/X.shape[1], index=X.columns)), None, None

    scaler = StandardScaler()
    Z = scaler.fit_transform(Xc.values)

    ncomp = min(ncomp, Xc.shape[1])
    pca = PCA(n_components=ncomp)
    pca.fit(Z)

    load = pd.DataFrame(pca.components_.T, index=Xc.columns)
    evr = pca.explained_variance_ratio_

    contrib = (load.abs().values @ evr.reshape(-1,1)).ravel()
    w_part = pd.Series(contrib, index=Xc.columns)

    # back to full 15 columns
    w = pd.Series(0.0, index=X.columns)
    w.loc[w_part.index] = w_part

    return _normalize_weights(w), load, evr


In [16]:
w_pca, loadings, evr = pca_metric_weights_relaxed(X, ncomp=4)

print("PCA weights:")
print(w_pca.round(4))

w_final = shrink_to_equal(w_pca, lam=0.3)
print("\nFinal shrunk weights:")
print(w_final.round(4))


PCA weights:
mdd_percent               0.0941
ldd_percent               0.0859
sharpe                    0.0911
cdd_percent               0.0925
mdd_days                  0.0753
ldd_days                  0.0831
pnl_t                     0.0945
days_since_first_trade    0.0000
drawdown_beta             0.0698
drawup_beta               0.0610
excess_mean_ret           0.0945
excess_ret                0.0896
cvar_5                    0.0000
success_rate              0.0686
dtype: float64

Final shrunk weights:
mdd_percent               0.0782
ldd_percent               0.0758
sharpe                    0.0773
cdd_percent               0.0777
mdd_days                  0.0726
ldd_days                  0.0749
pnl_t                     0.0784
days_since_first_trade    0.0500
drawdown_beta             0.0709
drawup_beta               0.0683
excess_mean_ret           0.0784
excess_ret                0.0769
cvar_5                    0.0500
success_rate              0.0706
dtype: float64


In [17]:
w_pca, loadings, evr = pca_metric_weights_relaxed(X, ncomp=4)

print("PCA weights:")
print(w_pca.round(4))

w_final = shrink_to_equal(w_pca, lam=0.3)
print("\nFinal shrunk weights:")
print(w_final.round(4))


PCA weights:
mdd_percent               0.0941
ldd_percent               0.0859
sharpe                    0.0911
cdd_percent               0.0925
mdd_days                  0.0753
ldd_days                  0.0831
pnl_t                     0.0945
days_since_first_trade    0.0000
drawdown_beta             0.0698
drawup_beta               0.0610
excess_mean_ret           0.0945
excess_ret                0.0896
cvar_5                    0.0000
success_rate              0.0686
dtype: float64

Final shrunk weights:
mdd_percent               0.0782
ldd_percent               0.0758
sharpe                    0.0773
cdd_percent               0.0777
mdd_days                  0.0726
ldd_days                  0.0749
pnl_t                     0.0784
days_since_first_trade    0.0500
drawdown_beta             0.0709
drawup_beta               0.0683
excess_mean_ret           0.0784
excess_ret                0.0769
cvar_5                    0.0500
success_rate              0.0706
dtype: float64


In [ ]:
#another way of calculating rankings


In [ ]:
# 1) rolling_metrics finished
expandings = expanding_window(df, min_len=200)

expandings_metrics = compute_rolling_metrics(
    [df.loc[idx] for idx in expandings],
    ret
)


In [14]:
# 1) rolling_metrics finished
print("计算expanding windows...")
expandings = expanding_window(df, 200, chunk=True)  # 使用chunk=True获取实际数据窗口
print(f"共有 {len(expandings)} 个expanding windows")

print("计算rolling metrics...")
expandings_metrics = compute_rolling_metrics(expandings, ret)
print("Rolling metrics 计算完成!")

##############################################
# 2) build X = final snapshot metrics (#agents × #metrics)
##############################################
print("构建最终指标矩阵...")
last_ts = list(expandings_metrics.values())[0].index.max()

rows=[]
for strat, dfm in expandings_metrics.items():
    rows.append(dfm.loc[last_ts].tolist())

X = pd.DataFrame(rows, index=list(expandings_metrics.keys()),
                 columns=dfm.columns)
print(f"指标矩阵形状: {X.shape}")

##############################################
# 3) PCA → w_pca
##############################################
print("计算PCA权重...")
w_pca, loadings, evr = pca_metric_weights_relaxed(X, ncomp=4)

print("PCA weights:")
print(w_pca.sort_values(ascending=False))

##############################################
# 4) EI using PCA weights
##############################################
print("计算EI...")
EIs_pca = compute_EI(expandings_metrics, alpha=0.02, weights=w_pca)
print("EI计算完成!")

计算expanding windows...
共有 51 个expanding windows
计算rolling metrics...
共有 51 个expanding windows
计算rolling metrics...
Rolling metrics 计算完成!
构建最终指标矩阵...
指标矩阵形状: (266, 14)
计算PCA权重...
PCA weights:
success_rate              0.087467
excess_mean_ret           0.084493
drawdown_beta             0.084475
drawup_beta               0.084211
excess_ret                0.081600
sharpe                    0.081535
mdd_percent               0.079650
ldd_days                  0.079381
cvar_5                    0.077872
cdd_percent               0.077614
ldd_percent               0.068730
mdd_days                  0.063349
pnl_t                     0.049621
days_since_first_trade    0.000000
dtype: float64
计算EI...
Rolling metrics 计算完成!
构建最终指标矩阵...
指标矩阵形状: (266, 14)
计算PCA权重...
PCA weights:
success_rate              0.087467
excess_mean_ret           0.084493
drawdown_beta             0.084475
drawup_beta               0.084211
excess_ret                0.081600
sharpe                    0.081535
mdd_percen

In [ ]:

##############################################
# 5) final score = area under EI curve
def auc_scores(EI):
    # EI: DataFrame (time × category)
    return EI.sum(axis=0)     # 行累加 (over time)

category_EI = compute_category_EI(EI_pca, category_map)

auc_by_cat = auc_scores(category_EI)   # each category
final_auc = auc_by_cat.mean()          # category equally weighted
print(final_auc)



In [11]:
def EI_AUC_ranking(expandings_metrics, pca_weights, alpha=0.02):
    """
    基于EI的AUC排名
    """
    print("计算EI时间序列...")
    # 1) EI(t) 时间序列
    EIs = compute_EI(expandings_metrics, alpha=alpha, weights=pca_weights)

    print("计算AUC...")
    # 2) 对每个 strategy 把 EI(t) 做面积
    auc = EIs.sum(axis=0)        # axis=0  → across time index

    # 3) 排序
    rank = auc.sort_values(ascending=False)

    return EIs, auc, rank

In [15]:
print("开始最终排名计算...")
EIs, auc, rank = EI_AUC_ranking(expandings_metrics, w_pca)

print("=== 最终排名 (前20名) ===")
print(rank.head(20))

print(f"\n总共处理了 {len(rank)} 个策略")
print(f"最高分: {rank.iloc[0]:.4f}")
print(f"最低分: {rank.iloc[-1]:.4f}")

开始最终排名计算...
计算EI时间序列...
计算AUC...
=== 最终排名 (前20名) ===
108_short    24.074397
139_short    22.527488
124_short    22.379422
218_long     22.337655
147_short    22.295649
220_short    22.288339
144_long     22.189874
101_short    21.894804
45_short     21.804790
197_short    21.669661
20_short     21.471503
174_short    21.413031
265_short    21.357960
22_short     21.273482
58_long      21.236179
189_short    21.170791
86_short     21.128997
68_long      20.868132
125_short    20.801587
111_short    20.800026
dtype: float64

总共处理了 266 个策略
最高分: 24.0744
最低分: 0.5845
计算AUC...
=== 最终排名 (前20名) ===
108_short    24.074397
139_short    22.527488
124_short    22.379422
218_long     22.337655
147_short    22.295649
220_short    22.288339
144_long     22.189874
101_short    21.894804
45_short     21.804790
197_short    21.669661
20_short     21.471503
174_short    21.413031
265_short    21.357960
22_short     21.273482
58_long      21.236179
189_short    21.170791
86_short     21.128997
68_long     